In [3]:
%pip install python-dotenv

In [6]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

Enter your Google API Key: ··········


In [7]:
!pip install -U langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.9
    Uninstalling langchain-core-1.2.9:
      Successfully uninstalled langchain-core-1.2.9
  Attempting uninstall: langgraph
    Found existing installation: langgraph 1.0.7
    Uninstalling langgraph-1.0.7:
      Successfully uninstalled langgraph-1.0.7
  Attempting uninstall: langchain
    Found existing installation: langchain 1.2.8
    Uninstalling langchain-1.2.8:
      Successfully uninstalled langchain-1.2.8


In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

#Model A: The "Accountant" (Precision)
llm_focused = ChatGoogleGenerativeAI(
  model="gemini-2.5-flash",
  temperature=0.0
)

#Model B: The "Poet" (Creativity)
llm_creative= ChatGoogleGenerativeAI(
  model="gemini-2.5-flash",
  temperature=1.0
)

In [9]:
prompt = "Define the word 'Idea' in one sentence."

print("--- FOCUSED (Temp=0) ---")
print(f"Run 1: {llm_focused.invoke(prompt).content}")
print(f"Run 2: {llm_focused.invoke(prompt).content}")

--- FOCUSED (Temp=0) ---
Run 1: An idea is a thought, concept, or mental image formed in the mind.
Run 2: An idea is a thought, concept, or mental image formed in the mind.


In [10]:
print("--- CREATIVE (Temp=1) ---")
print(f"Run 1: {llm_creative.invoke(prompt).content}")
print(f"Run 2: {llm_creative.invoke(prompt).content}")

--- CREATIVE (Temp=1) ---
Run 1: An idea is a thought, concept, or mental impression that forms in the mind, often serving as a basis for understanding, action, or creation.
Run 2: An idea is a mental construct, such as a thought, concept, plan, or suggestion, formed in the mind.


# **Unit 2 - Part 1b: Prompts & Parsers**

In [11]:
# Setup from Part 1a (Hidden for brevity)
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [12]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You are a rude teenager. You use slang and don't care about grammar."),
    HumanMessage(content="What is the capital of France?")
]

response = llm.invoke(messages)
print(response.content)

Ugh, like, it's Paris, duh. Like, everyone knows that. Are you serious right now?


In [13]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a translator. Translate {input_language} to {output_language}."),
    ("human", "{text}")
])
print(f"Required variables: {template.input_variables}")

Required variables: ['input_language', 'output_language', 'text']


In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

raw_msg = llm.invoke("Hi")
print(f"Raw Type: {type(raw_msg)}")

clean_text = parser.invoke(raw_msg)
print(f"Parsed Type: {type(clean_text)}")
print(f"Content: {clean_text}")

Raw Type: <class 'langchain_core.messages.ai.AIMessage'>
Parsed Type: <class 'langchain_core.messages.base.TextAccessor'>
Content: Hi there! How can I help you today?


# **Unit 2-Part 1c: LCEL(LangChain Expression Language)**

In [15]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
template = ChatPromptTemplate.from_template("Tell me a fun fact about {topic}.")
parser = StrOutputParser()

In [16]:
prompt_value = template.invoke({"topic": "Crows"})

response_obj = llm.invoke(prompt_value)

final_text = parser.invoke(response_obj)

print(final_text)

Here's a fun one:

Crows are incredibly intelligent and have an amazing memory, especially when it comes to people! They can recognize and remember individual human faces for years, and even pass that information on to other crows in their flock. So, if you're nice to a crow, they (and potentially their friends) might remember you fondly; if you're mean, well, they might hold a grudge!


In [17]:
chain = template | llm | parser

print(chain.invoke({"topic": "Octopuses"}))

Here's a fun fact about octopuses:

Octopuses have **three hearts** and **blue blood**!

Two hearts pump blood through their gills, and the third circulates it to the rest of their body. Their blood is blue because it uses a copper-rich protein called hemocyanin to transport oxygen, instead of the iron-rich hemoglobin that makes our blood red.


# Assignment

In [18]:
template = ChatPromptTemplate.from_template(
    """
    The current year is 2025.
    What year was the movie '{movie}' released?
    Then calculate how many years ago that was.
    Provide the answer in one sentence.
    """
)


In [19]:
chain = template | llm | StrOutputParser()

In [20]:
print(chain.invoke({"movie": "Inception"}))

The movie 'Inception' was released in 2010, which was 15 years ago.
